# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from statistics import mean


In [22]:
data = pd.read_csv('/datasets/Churn.csv')
data.shape

(10000, 14)

In [23]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Удаляем первые три столбца, т.к. они не содержат полезной для нас информации.

In [25]:
data = data.drop(['RowNumber','CustomerId', 'Surname'], axis = 1)
data = data.dropna()
data.shape

(9091, 11)

Разбиваем данные на выборки.

In [26]:
features = data.drop('Exited', axis=1)
target = data['Exited']

Переводим категориальные признаки в численные.

In [27]:
features = pd.get_dummies(features, columns=['Geography', 'Gender'], drop_first=True)
features.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,1,0


In [28]:
target.mean()

0.2039379606203938

In [29]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, train_size=0.75, random_state=12345, stratify=target)

In [30]:
target_train.mean()

0.20387210325608682

In [31]:
target_test.mean()

0.20413550373955125

Стандартизируем численные признаки. Отдельно обучающую и тестовую выборки, чтобы избежать "утечки".

In [32]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
pd.options.mode.chained_assignment = None
features_train[numeric] = scaler.fit_transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Исследование задачи

### Исследуем баланс классов.

In [33]:
target_train.value_counts(normalize=True)

0    0.796128
1    0.203872
Name: Exited, dtype: float64

Положительных классов всего 20%.
Построим модели без учета дисбаланса классов.

### Решающее дерево.

Подбираем оптимальные гиперпараметры для модели по f1-мере.

In [34]:
columns = {'DecisionTree': [], 'RandomForest': [], 'LogisticRegression': []}
param_grid = {'max_depth': range(2, 30, 2)}
model = DecisionTreeClassifier(random_state=12345)
model_tree = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs = -1)
model_tree.fit(features_train, target_train)
model_tree.best_score_.round(3)

0.549

In [35]:
model_tree.best_params_

{'max_depth': 8}

In [36]:
columns['DecisionTree'].append(model_tree.best_score_.round(3))

Смотрим метрику AUC-ROC для модели с оптимальными гиперпараметрами.

In [37]:
model = DecisionTreeClassifier(max_depth=8, random_state=12345)
scores_list = cross_val_score(model, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='roc_auc',
                              n_jobs = -1
                              )

scores_list.mean()

0.8125886049822972

In [38]:
columns['DecisionTree'].append(scores_list.mean().round(3))

### Случайный лес.

Подбираем оптимальные гиперпараметры для модели по f1-мере.

In [39]:
param_grid = {'max_depth': range(2, 30, 2), 'n_estimators': range(10, 100, 10)}
model = RandomForestClassifier(random_state=12345)
model_forest = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs = -1)
model_forest.fit(features_train, target_train)
model_forest.best_score_

0.5799722524275941

In [40]:
model_forest.best_params_

{'max_depth': 24, 'n_estimators': 70}

Смотрим метрику AUC-ROC для модели с оптимальными гиперпараметрами.

In [41]:
model = RandomForestClassifier(max_depth=24, n_estimators=70, random_state=12345)
scores_list = cross_val_score(model, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='roc_auc',
                              n_jobs = -1
                              )

scores_list.mean()

0.8468738152538794

In [42]:
columns['RandomForest'].append(model_forest.best_score_.round(3))
columns['RandomForest'].append(scores_list.mean().round(3))

### Логистическая регрессия.

Смотрим метрику AUC-ROC

In [43]:
model_log = LogisticRegression(random_state=12345, solver='liblinear')
scores_list = cross_val_score(model_log, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='f1',
                              n_jobs = -1
                              )

scores_list.mean()

0.29561432158396106

In [44]:
columns['LogisticRegression'].append(scores_list.mean().round(3))

In [45]:
scores_list = cross_val_score(model_log, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='roc_auc',
                              n_jobs = -1
                              )

scores_list.mean()

0.7619577197167338

In [46]:
columns['LogisticRegression'].append(scores_list.mean().round(3))

Наибольшее значение AUC-ROC показала модель "Случайный лес".

## Борьба с дисбалансом

### Взвешивание классов.

#### Модель "Решающее дерево"

In [47]:
param_grid = {'max_depth': range(2, 30, 2)}
model = DecisionTreeClassifier(class_weight='balanced', random_state=12345)
model_tree = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs = -1)
model_tree.fit(features_train, target_train)
model_tree.best_score_

0.5703341890411309

In [48]:
model_tree.best_params_

{'max_depth': 6}

In [49]:
model = DecisionTreeClassifier(class_weight='balanced', max_depth=6, random_state=12345)
scores_list = cross_val_score(model, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='roc_auc',
                              n_jobs = -1
                              )

scores_list.mean()

0.8285241736595195

In [50]:
columns['DecisionTree'].append(model_tree.best_score_.round(3))
columns['DecisionTree'].append(scores_list.mean().round(3))

#### Модель "Случайный лес"

In [51]:
param_grid = {'max_depth': range(2, 30, 2), 'n_estimators': range(10, 100, 10)}
model = RandomForestClassifier(class_weight='balanced', random_state=12345)
model_forest = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs = -1)
model_forest.fit(features_train, target_train)
model_forest.best_score_

0.618778907294272

In [52]:
model_forest.best_params_

{'max_depth': 8, 'n_estimators': 90}

In [53]:
model = RandomForestClassifier(class_weight='balanced', max_depth=8, n_estimators=90, random_state=12345)
scores_list = cross_val_score(model, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='roc_auc',
                              n_jobs = -1
                              )

scores_list.mean()

0.8572419681180993

In [54]:
columns['RandomForest'].append(model_forest.best_score_.round(3))
columns['RandomForest'].append(scores_list.mean().round(3))

#### Логистическая регрессия

In [55]:
model_log = LogisticRegression(class_weight='balanced', random_state=12345, solver='liblinear')
scores_list = cross_val_score(model_log, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='f1',
                              n_jobs = -1
                              )

scores_list.mean()

0.4932796488062127

In [56]:
columns['LogisticRegression'].append(scores_list.mean().round(3))

In [57]:
scores_list = cross_val_score(model_log, 
                              X=features_train, 
                              y=target_train, 
                              cv=5,
                              scoring='roc_auc',
                              n_jobs = -1
                              )

scores_list.mean()

0.7650655764819618

In [58]:
columns['LogisticRegression'].append(scores_list.mean().round(3))

### Увеличение выборки.

Увеличим положительные объекты обучающей выборки в 4 раза, тем самым сбалансируем классы.

In [59]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

Напишем функцию, которая разбивает данные на обучающий и тренировочный фолды. А затем на каждом фолде апсемплит наименьший класс на repeat в цикле от 1 до 5, обучает модель и вычисляет F1-меру и AUC_ROC для каждой repeat. Возвращает словарь с метриками.

In [60]:
def score_model(model, params):
    
    cv = KFold(n_splits=5, shuffle=False)

    result = {}
    

    for repeat in np.arange(1, 5):
        f1_list = []
        auc_roc_list =[]
        for train_fold_index, val_fold_index in cv.split(features_train, target_train):
            features_train_fold = features_train.iloc[train_fold_index]
            target_train_fold = target_train.iloc[train_fold_index]
            features_val_fold = features_train.iloc[val_fold_index]
            target_val_fold = target_train.iloc[val_fold_index]
            
            features_train_fold_upsample, target_train_fold_upsample = upsample(features_train_fold,
                                                                                target_train_fold,
                                                                                repeat)
            model_obj = model(**params).fit(features_train_fold_upsample, target_train_fold_upsample)
        
            f1 = f1_score(target_val_fold, model_obj.predict(features_val_fold))
            f1_list.append(f1)
        
            auc_roc = roc_auc_score(target_val_fold,  model_obj.predict_proba(features_val_fold)[:, 1])
            auc_roc_list.append(auc_roc)
            
        result.update({repeat:{'f1':f1_list, 'auc_roc': auc_roc_list}})
        
    return result

In [61]:
example_params = {'max_depth': 6, 'random_state': 12345}
result = score_model(DecisionTreeClassifier, example_params)
result

{1: {'f1': [0.5438972162740899,
   0.5480769230769231,
   0.5454545454545455,
   0.5754716981132075,
   0.5106382978723404],
  'auc_roc': [0.8123809523809524,
   0.8440444559980824,
   0.8277070678815005,
   0.8470028125105893,
   0.8319706287596196]},
 2: {'f1': [0.5847176079734219,
   0.5795454545454546,
   0.5948717948717949,
   0.575609756097561,
   0.5921985815602837],
  'auc_roc': [0.8123402255639098,
   0.8334706265635392,
   0.8228483988704178,
   0.8519501202941276,
   0.8345632960311858]},
 3: {'f1': [0.5718270571827057,
   0.5718390804597702,
   0.5817655571635312,
   0.581021897810219,
   0.5721997300944669],
  'auc_roc': [0.8104323308270678,
   0.837478013389466,
   0.8274339966965046,
   0.8478804513571212,
   0.8223880095439728]},
 4: {'f1': [0.572972972972973,
   0.559892328398385,
   0.5830903790087463,
   0.5764075067024129,
   0.5800273597811217],
  'auc_roc': [0.8080889724310777,
   0.8365934848735497,
   0.8229466378942881,
   0.8438548337907898,
   0.8267214436939

In [62]:
for i in range(1, 5):
    print(i, mean(result[i]['f1']), mean(result[i]['auc_roc']))

1 0.5447077361582213 0.8326211835061489
2 0.5853886390097032 0.8310345334646361
3 0.5757306645421386 0.8291225603628265
4 0.5744781093727278 0.8276410745367252


#### Решающее дерево

In [63]:
%%time

score = 0.5
for depth in range(2, 30, 2):
    example_params = {'max_depth': depth, 'random_state': 12345}
    result = score_model(DecisionTreeClassifier, example_params)
    for i in range(1, 5):
        #print(i, mean(result[i]['f1']), mean(result[i]['auc_roc']))
        if mean(result[i]['f1']) > score:
            score = mean(result[i]['f1'])
            best_params = example_params
            best_params['f1'] = mean(result[i]['f1'])
            best_params['auc_roc'] = mean(result[i]['auc_roc'])
            best_params['repeat'] = i
print(best_params)

{'max_depth': 6, 'random_state': 12345, 'f1': 0.5853886390097032, 'auc_roc': 0.8310345334646361, 'repeat': 2}
CPU times: user 9.27 s, sys: 23.9 ms, total: 9.3 s
Wall time: 9.31 s


In [64]:
columns['DecisionTree'].append(best_params['f1'].round(3))
columns['DecisionTree'].append(best_params['auc_roc'].round(3))

#### Случайный лес

In [67]:
score = 0.5
for depth in range(2, 30, 2):
    for est in range(10, 100, 10):
        example_params = {'max_depth': depth, 'n_estimators': est, 'random_state': 12345}
        result = score_model(RandomForestClassifier, example_params)
        for i in range(1, 5):
            #print(i, mean(result[i]['f1']), mean(result[i]['auc_roc']))
            if mean(result[i]['f1']) > score:
                score = mean(result[i]['f1'])
                best_params = example_params
                best_params['f1'] = mean(result[i]['f1'])
                best_params['auc_roc'] = mean(result[i]['auc_roc'])
                best_params['repeat'] = i
print(best_params)

{'max_depth': 12, 'n_estimators': 60, 'random_state': 12345, 'f1': 0.6182205868861036, 'auc_roc': 0.8510547704030835, 'repeat': 4}


Реализуем апсемплинг через SMOTE.

In [72]:
#%%time

#from imblearn.pipeline import Pipeline, make_pipeline
#from imblearn.over_sampling import SMOTE
    
#imba_pipeline = make_pipeline(SMOTE(random_state=12345), RandomForestClassifier(random_state=12345))
#score = cross_val_score(imba_pipeline, features_train, target_train, scoring='f1', cv=5)
#param_grid = {'randomforestclassifier__max_depth': range(2, 30, 2),
#              'randomforestclassifier__n_estimators': range(10, 100, 10),
#              'randomforestclassifier__n_jobs': [-1]
#             }
#grid_imba = GridSearchCV(imba_pipeline, param_grid, cv=5, scoring='f1', n_jobs = -1)
#grid_imba.fit(features_train, target_train)
#grid_imba.best_score_


In [73]:
#grid_imba.best_params_

In [74]:
columns['RandomForest'].append(best_params['f1'].round(3))
columns['RandomForest'].append(best_params['auc_roc'].round(3))

#### Логистическая регрессия

In [76]:
score = 0.1
example_params = {'random_state': 12345, 'solver': 'liblinear'}
result = score_model(LogisticRegression, example_params)
for i in range(1, 5):    
    if mean(result[i]['f1']) > score:
        score = mean(result[i]['f1'])
        best_params = example_params
        best_params['f1'] = mean(result[i]['f1'])
        best_params['auc_roc'] = mean(result[i]['auc_roc'])
        best_params['repeat'] = i
print(best_params)

{'random_state': 12345, 'solver': 'liblinear', 'f1': 0.4932254263734392, 'auc_roc': 0.7646923726993997, 'repeat': 4}


In [78]:
columns['LogisticRegression'].append(best_params['f1'].round(3))
columns['LogisticRegression'].append(best_params['auc_roc'].round(3))

Оба варианта дали одинаковый результат.

## Тестирование модели

Протестируем модель на тестовой выборке по двум метрикам - F1-мера и AUC-ROC.

In [79]:
predicted_test = model_forest.predict(features_test)
f1_score(target_test, predicted_test).round(3)

0.612

In [80]:
roc_auc_score(target_test, model_forest.predict_proba(features_test)[:, 1])

0.8650402203541677

Проверим модель на адекватность. 

In [81]:
dummy_clf = DummyClassifier(strategy="uniform", random_state=12345)
dummy_clf.fit(features_train, target_train)
predicted_dummy = dummy_clf.predict(features_test)
roc_auc_score(target_test, dummy_clf.predict_proba(features_test)[:, 1])

0.5

## Вывод

In [82]:
index_list = ['Без учета дисбаланса F1',
              'Без учета дисбаланса AUC_ROC',
              'Взвешивание классов F1',
              'Взвешивание классов AUC_ROC',
              'Апсемплинг F1',
             'Апсемплинг AUC_ROC']

pd.DataFrame(index = index_list, data = columns)

,DecisionTree,RandomForest,LogisticRegression
Без учета дисбаланса F1,0.549,0.580,0.296
Без учета дисбаланса AUC_ROC,0.813,0.847,0.762
Взвешивание классов F1,0.570,0.619,0.493
Взвешивание классов AUC_ROC,0.829,0.857,0.765
Апсемплинг F1,0.585,0.618,0.493
Апсемплинг AUC_ROC,0.831,0.851,0.765


В проекте было исследовано 3 модели по двум метрикам: F1-мера и AUC-ROC.

Без учета дисбаланса классов наилучщий результат показала модель случайный лес.

Борьбу с дисбалансом провели двумя методами: взвешиванием классов и апсемплингом.

Метод взвешивания классов на модели случайный лес дал максимальные метрики.